In [1]:
# this is multi-criteria ABC analysis
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.float_format', '{:.4f}'.format)
df = pd.read_csv('data.txt', encoding='windows-1251')

In [3]:
summed = df.groupby(['DR_NDrugs']).agg({'DR_Kol': 'sum', 'DR_CRoz': 'sum'})
summed['amnt_share'] = summed.DR_Kol / sum(summed['DR_Kol'])
summed['retail_share'] = summed.DR_CRoz / sum(summed['DR_CRoz'])

In [4]:
summed = summed.sort_values('amnt_share', ascending=False)
summed

,DR_Kol,DR_CRoz,amnt_share,retail_share
DR_NDrugs,,,,
ПАКЕТ,126.0000,248.0000,0.0275,0.0002
"ЦЕФТРИАКСОН 1Г. №1 ПОР. Д/Р-РА Д/В/В,В/М ФЛ. /ЛЕККО/",60.0000,2280.0000,0.0131,0.0017
"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /ЛЕККО/",48.0000,552.0000,0.0105,0.0004
"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",40.0000,28.0000,0.0087,0.0000
"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТАДА/",35.0000,4215.0000,0.0076,0.0031
...,...,...,...,...
"НАТРИЯ ХЛОРИД 0,9% 400МЛ. №16 Р-Р Д/ИНФ. КОНТ. /МОСФАРМ/",0.0625,669.0000,0.0000,0.0005
"СФМ ШПРИЦ 2МЛ. 3-Х КОМП. 0,63Х32ММ 23G №100 [SFM]",0.0500,918.0000,0.0000,0.0007
ДЕРМАГРИП ХАЙ РИСК ПЕРЧАТКИ ЛАТ. СМОТР. Н/СТЕР. Н/ОПУДР Р.L №50 (25ПАР),0.0400,2000.0000,0.0000,0.0015


In [5]:
summed['amnt_cumsum'] = summed['amnt_share'].cumsum()
summed['retail_cumsum'] = summed['retail_share'].cumsum()
summed['ABC_amnt'] = np.where(summed['amnt_cumsum'] < 0.8, 'A', np.where(summed['amnt_cumsum'] < 0.95, 'B', 'C'))
summed['ABC_retail'] = np.where(summed['retail_cumsum'] < 0.8, 'A', np.where(summed['retail_cumsum'] < 0.95, 'B', 'C'))
summed = summed.reset_index()

In [6]:
# here i indexed df with products that have different groups
diff_groups =  summed[summed['ABC_amnt'] != summed['ABC_retail']]
diff_groups

,DR_NDrugs,DR_Kol,DR_CRoz,amnt_share,retail_share,amnt_cumsum,retail_cumsum,ABC_amnt,ABC_retail
938,ЭДАРБИ 80МГ. №28 ТАБ.,1.0000,1239.0000,0.0002,0.0009,0.8001,0.6115,B,A
939,ЭГИЛОК 50МГ. №60 ТАБ. /ЭГИС/,1.0000,133.0000,0.0002,0.0001,0.8004,0.6116,B,A
940,ЭГИЛОК 100МГ. №30 ТАБ. /ЭГИС/,1.0000,130.0000,0.0002,0.0001,0.8006,0.6117,B,A
941,ЭВО ПАНТЕНОЛ МОЛОЧКО Д/ТЕЛА 150МЛ. [EVO] /АВАНТА/,1.0000,165.0000,0.0002,0.0001,0.8008,0.6118,B,A
942,"ЭВКАЗОЛИН АКВА 0,1% 10МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ФАР...",1.0000,114.0000,0.0002,0.0001,0.8010,0.6119,B,A
...,...,...,...,...,...,...,...,...,...
1818,"КО-ПЕРИНЕВА 1,25МГ.+4МГ. №90 ТАБ. /KRKA/",1.0000,1353.0000,0.0002,0.0010,0.9920,0.9479,C,B
1819,"КО-ПЕРИНЕВА 1,25МГ.+4МГ. №30 ТАБ. /KRKA/",1.0000,683.0000,0.0002,0.0005,0.9923,0.9484,C,B
1820,"КО-ДАЛЬНЕВА 5МГ+2,5МГ.+8МГ. №90 ТАБ.",1.0000,1658.0000,0.0002,0.0012,0.9925,0.9496,C,B
1821,КЛОТРИМАЗОЛ-ТЕВА 1% 30Г. КРЕМ Д/НАРУЖ.ПРИМ. ТУБА,1.0000,292.0000,0.0002,0.0002,0.9927,0.9498,C,B


In [8]:
# this is how much does each difference appear
diff_groups.value_counts(subset=['ABC_amnt', 'ABC_retail'])

ABC_amnt  ABC_retail
B         A             467
C         B             197
Name: count, dtype: int64